# Extração da característica de relação sinal-ruído de dados de EEG

A ideia é utilizar dados fictícios de ruído e sinal "bom"., para criarmos a relação dos dois sinais e obter como resultado um sinal de interesse "limpo".

A partir deste sinal, podemos no contexto de caracterização de foco, ainda extrair os rítmos cerebrais ou então classificar sinais com a presença ou não de foco, de forma que as amostras de sinais extraídas de um buffer sejam rotuladas com com a presença ou não de foco.

Esta atividade pode ser realizada em conjunto com um classificador comumente utilizado, como é o caso do SVM. Neste caso, uma porcetagem das amostras são utilizadas para treino e o restante para teste (p.e. 30 e 70% respectivamente).

In [61]:
import numpy as np
import mne
from scipy.signal import welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [62]:
# Definição do info (MNE)
n_channels = 8
ch_types = ['eeg'] * n_channels
sfreq = 250
ch_names = ["F3", "Fz", "F4", "C3", "Cz", "C4", "P3", "P4"]
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
info.set_montage("standard_1020")

<Info | 8 non-empty values
 bads: []
 ch_names: F3, Fz, F4, C3, Cz, C4, P3, P4
 chs: 8 EEG
 custom_ref_applied: False
 dig: 11 items (3 Cardinal, 8 EEG)
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 8
 projs: []
 sfreq: 250.0 Hz
>

In [67]:
start_time = 0.35  # início aos 0,35 segundos
end_time = 18 * 60  # fim após 19 minutos (18 min + 1 min)
test_time = 200  # teste escrito de 0 a 200 segundos (3 min 20s)

# Carregar os dados dos arquivos reais
glucamon_files = [
    '../database/s3/glucamon/OpenBCI-RAW-2023-11-08_13-55-20.txt',
    '../database/s3/glucamon/OpenBCI-RAW-2023-11-08_13-57-31.txt'
]
glucamon_data = [np.loadtxt(f, delimiter=',', skiprows=5, usecols=range(1, 9)) for f in glucamon_files]
glucamon_data_combined = np.vstack(glucamon_data)

# Cortando os dados de glucamon
data_glucamon_cut = glucamon_data_combined[int(start_time * sfreq):int(end_time * sfreq), :]  # Cortar de 0,35s até 19min
data_glucamon_test = glucamon_data_combined[:int(test_time * sfreq), :] 

insulina_files = [
    '../database/s3/insulina/OpenBCI-RAW-2023-10-24_14-19-06.txt',
    '../database/s3/insulina/OpenBCI-RAW-2023-10-24_14-19-27.txt',
    '../database/s3/insulina/OpenBCI-RAW-2023-10-24_14-20-06.txt',
    '../database/s3/insulina/OpenBCI-RAW-2023-10-24_14-21-23.txt',
    '../database/s3/insulina/OpenBCI-RAW-2023-10-24_14-28-10.txt'
]
insulina_data = [np.loadtxt(f, delimiter=',', skiprows=5, usecols=range(1, 9)) for f in insulina_files]
insulina_data_combined = np.vstack(insulina_data)

# Teste final
teste_final_file = '../database/s3/teste/OpenBCI-RAW-2023-11-24_12-19-35.txt'
teste_final_data = np.loadtxt(teste_final_file, delimiter=',', skiprows=5, usecols=range(1, 9))

# Estimativa do ruído usando o período basal
data_basal = teste_final_data[:int(60 * sfreq), :]

In [68]:
# Estimar o ruído basal (SNR Basal)
def estimate_noise_power(data_basal):
    noise_power = []
    for channel_data in data_basal.T:  
        fft_result = np.fft.fft(channel_data)
        psd = np.abs(fft_result) ** 2 
        noise_power.append(np.mean(psd)) 
    return noise_power

# Função para calcular a relação sinal-ruído (SNR)
def calculate_snr(data, noise_power):
    snr_values = []
    for channel_data, noise in zip(data.T, noise_power): 
        fft_result = np.fft.fft(channel_data)
        psd = np.abs(fft_result) ** 2
        signal_power = np.mean(psd)
        snr = 10 * np.log10(signal_power / noise)
        snr_values.append(snr)
    return snr_values

# Calcular a relação sinal-ruído para cada condição
noise_power = estimate_noise_power(data_basal)
snr_glucamon = calculate_snr(data_glucamon_cut, noise_power)
snr_insulina = calculate_snr(insulina_data_combined, noise_power)
snr_teste_final = calculate_snr(teste_final_data, noise_power)


In [69]:
def extract_advanced_features(data, window_size=5):
    n_windows = data.shape[0] // (sfreq * window_size)
    snr_median = []
    snr_mean = []
    snr_variance = []
    for i in range(n_windows):
        window_data = data[i * sfreq * window_size:(i + 1) * sfreq * window_size, :]
        snr_values = calculate_snr(window_data, noise_power)
        snr_median.append(np.median(snr_values))
        snr_mean.append(np.mean(snr_values))
        snr_variance.append(np.var(snr_values))
    return snr_mean, snr_median, snr_variance


mean_glucamon, median_glucamon, var_glucamon = extract_advanced_features(data_glucamon_cut)
mean_insulina, median_insulina, var_insulina = extract_advanced_features(insulina_data_combined)
mean_teste_final, median_teste_final, var_teste_final = extract_advanced_features(teste_final_data)

X_features = np.array(mean_glucamon + mean_insulina + mean_teste_final +
                      median_glucamon + median_insulina + median_teste_final +
                      var_glucamon + var_insulina + var_teste_final).reshape(-1, 3)

y_labels = np.array([0] * len(mean_glucamon) + [1] * len(mean_insulina) + [2] * len(mean_teste_final))

X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.3, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Acurácia do modelo: {accuracy}")
print("Relatório de classificação:")
print(report)

Acurácia do modelo: 0.8785714285714286
Relatório de classificação:
              precision    recall  f1-score   support

           0       0.68      0.60      0.64        25
           1       0.87      0.91      0.89        75
           2       1.00      1.00      1.00        40

    accuracy                           0.88       140
   macro avg       0.85      0.84      0.84       140
weighted avg       0.87      0.88      0.88       140

